In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scan
import simulate_data

# h2 hippocampus

In [2]:
def get_data(clusters_file, spikes_file):
#     labels_file = open(labels_file)
    clusters = clusters_file.readlines()
    num_clu = int(clusters[0][:-1])-2
#     num_clu = clusters[0][:-1]
    clusters = np.array([k[:-1] for k in clusters],dtype='int')[1:]
#     labels = np.array([k[:-1] for k in labels])
    
    spikes = spikes_file.readlines()
    spikes = np.array([k[:-1] for k in spikes],dtype='float')
    
    spikes = spikes[clusters>1]
    clusters = clusters[clusters>1]
    
    spikes, clusters = spikes[spikes.argsort()], clusters[spikes.argsort()] # sorting the arrays by spike time
    
    
    return clusters, spikes, num_clu


In [5]:
file =r'C:\Users\miles\Projects\datasets\crcns\hc2\ec013.527\ec013.527.clu.1'
os.path.abspath('C:\\Users\\miles\\Projects\\datasets')

'C:\\Users\\miles\\Projects\\datasets'

In [6]:
shanks = 4
clusters, spikes, num_clu = [],[],[]

for i in range(shanks):
    (clusters_file, spikes_file) = (
        open(f"C:\\Users\\miles\\Projects\\datasets\\crcns\\hc2\\ec013.527\\ec013.527.clu.{i+1}"),
        open(f"C:\\Users\miles\\Projects\\datasets\\crcns\\hc2\\ec013.527\\ec013.527.res.{i+1}")
    )
    a,b,c = get_data(clusters_file, spikes_file)
    clusters.append(a)
    spikes.append(b)
    num_clu.append(c)